Personal Finance: Expense Data Cleaner

In [12]:
#Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest

In [13]:
# GitHub raw CSV file
url = "https://raw.githubusercontent.com/NickCutrone/Expense_Data_Cleanup/refs/heads/main/sample_data.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(url)

# Display the first few rows
print(df.head())

         Date Merchant  Amount  Category
0  2025-01-11   Amazon  876.32  Shopping
1  2024-11-28   Amazon  284.17  Shopping
2  2024-10-10   Amazon  267.79  Shopping
3  2024-11-05   Amazon  252.19  Shopping
4  2024-11-15   Amazon  246.57  Shopping


In [14]:
# Display first few rows
print(df.head())

         Date Merchant  Amount  Category
0  2025-01-11   Amazon  876.32  Shopping
1  2024-11-28   Amazon  284.17  Shopping
2  2024-10-10   Amazon  267.79  Shopping
3  2024-11-05   Amazon  252.19  Shopping
4  2024-11-15   Amazon  246.57  Shopping


In [15]:
# Standardize merchant names
merchant_mapping = {
    'Water Bill': 'Water Bill',
    'Spotify': 'Spotify',
    'Netflix': 'Netflix',
    'Amazon': 'Amazon',
    'Apple Store': 'Apple Store',
    'Electricity Bill': 'Electricity Bill',
    'Lyft': 'Lyft',
    'Insurance': 'Insurance',
    'Gym Membership': 'Gym Membership',
    'Uber': 'Uber',
    'Phone Bill': 'Phone Bill',
    "McDonald's": "McDonald's",
    'Walmart': 'Walmart',
    'Restaurant': 'Restaurant',
    'Rent Payment': 'Rent Payment',
    'Shell Gas': 'Shell Gas',
    'Starbucks': 'Starbucks',
    'Best Buy': 'Best Buy',
    'BP Gas': 'BP Gas',
    'Target': 'Target'
}
df['Merchant'] = df['Merchant'].replace(merchant_mapping)

In [16]:
# Data quality checks
df['Duplicate'] = df.duplicated(subset=['Date', 'Merchant', 'Amount'])
df['Missing Values'] = df.isnull().sum(axis=1) > 0

In [17]:
# Categorize expenses
category_mapping = {
    'Amazon': 'Shopping',
    'Uber': 'Transport',
    'Lyft': 'Transport',
    'Starbucks': 'Food & Drink',
    "McDonald's": 'Food & Drink',
    'Restaurant': 'Food & Drink',
    'Netflix': 'Entertainment',
    'Spotify': 'Entertainment',
    'Apple Store': 'Shopping',
    'Target': 'Shopping',
    'Best Buy': 'Electronics',
    'Shell Gas': 'Gas',
    'BP Gas': 'Gas',
    'Walmart': 'Groceries',
    'Rent Payment': 'Housing',
    'Gym Membership': 'Health & Fitness',
    'Insurance': 'Bills',
    'Phone Bill': 'Bills',
    'Electricity Bill': 'Utilities',
    'Water Bill': 'Utilities'
}
df['Category'] = df['Merchant'].map(category_mapping)

In [18]:
df['Category'] = df['Merchant'].map(category_mapping)

# Isolate rows where Merchant or Category is missing
unmapped_data = df[df['Merchant'].isna() | df['Category'].isna()]

# Display unmapped data
print(unmapped_data)

           Date   Merchant   Amount Category  Duplicate  Missing Values
36   2024-10-14  AMZN-2243   244.14      NaN      False           False
397  2024-11-13    SB-3399    14.72      NaN      False           False
610  2024-12-26        NaN  2478.65      NaN      False            True
611  2024-10-03        NaN  2425.09      NaN      False            True
612  2024-10-11        NaN  2194.14      NaN      False            True
613  2025-01-21        NaN  1798.14      NaN      False            True
614  2024-11-03        NaN  1434.72      NaN      False            True
615  2024-09-21        NaN   975.37      NaN      False            True
616  2024-10-23        NaN   538.24      NaN      False            True
617  2024-11-30        NaN   394.30      NaN      False            True
618  2025-01-03        NaN   164.30      NaN      False            True
619  2024-11-09        NaN    65.15      NaN      False            True


In [19]:
# Detect anomalous expenses for Each Merchant and Category
for (merchant, category), group in df.groupby(['Merchant', 'Category']):
    if len(group) > 5:  # Ensure enough samples for isolation forest
        iso_forest = IsolationForest(contamination=0.05, random_state=42)
        anomaly_scores = iso_forest.fit_predict(group[['Amount']])
        df.loc[group.index, 'Anomaly_Score'] = anomaly_scores
        df.loc[group.index, 'Anomalous'] = anomaly_scores == -1
    else:
        df.loc[group.index, 'Anomalous'] = False

# Ensure 'Anomalous' column is boolean and handle NaN values
df['Anomalous'] = df['Anomalous'].fillna(False).astype(bool)

/var/folders/9w/gyqxr8zd0dx5qzc697rly9p00000gn/T/ipykernel_32253/3918141514.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Anomalous'] = df['Anomalous'].fillna(False).astype(bool)


In [20]:
# Display anomalous expenses
print(df[df['Anomalous']])

           Date          Merchant    Amount          Category  Duplicate  \
0    2025-01-11            Amazon    876.32          Shopping      False   
35   2024-10-30            Amazon     12.20          Shopping      False   
37   2024-10-24       Apple Store   2598.08          Shopping      False   
54   2024-11-26          Best Buy   8901.12       Electronics      False   
67   2024-09-21          Best Buy      0.02       Electronics      False   
77   2024-11-21            BP Gas     98.98               Gas      False   
82   2024-10-20            BP Gas    332.21               Gas      False   
129  2024-08-14  Electricity Bill    799.22         Utilities      False   
146  2025-01-01  Electricity Bill     42.03         Utilities      False   
163  2024-11-17    Gym Membership    500.00  Health & Fitness      False   
174  2024-10-07    Gym Membership     23.09  Health & Fitness      False   
175  2024-11-12         Insurance    494.50             Bills      False   
182  2024-09

In [21]:
# Save cleaned data to CSV
df.to_csv('cleaned_expenses.csv', index=False)

In [22]:
# Display final processed DataFrame
print(df.head())

         Date Merchant  Amount  Category  Duplicate  Missing Values  \
0  2025-01-11   Amazon  876.32  Shopping      False           False   
1  2024-11-28   Amazon  284.17  Shopping      False           False   
2  2024-10-10   Amazon  267.79  Shopping      False           False   
3  2024-11-05   Amazon  252.19  Shopping      False           False   
4  2024-11-15   Amazon  246.57  Shopping      False           False   

   Anomaly_Score  Anomalous  
0           -1.0       True  
1            1.0      False  
2            1.0      False  
3            1.0      False  
4            1.0      False  
